In [ ]:
# Importing libraries
import pickle
import os
import re
import gc
import datetime
import numpy as np
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
from google.cloud import bigquery
from matplotlib import pyplot as plt
import google.oauth2.credentials
from google.cloud import bigquery
from dateutil.relativedelta import relativedelta
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import seaborn as sns

# build model
import xgboost as xgb
from sklearn.metrics import roc_auc_score

In [ ]:
local_dev = True

In [ ]:
project_id = 'divg-groovyhoon-pr-d2eab4'
proj_id = 'divg-groovyhoon-pr-d2eab4'
dataset_id = 'telus_postpaid_12m_churn_v1'

In [ ]:
def get_gcp_bqclient(project_id, use_local_credential=False):
    token = os.popen('gcloud auth print-access-token').read()
    token = re.sub(f'\n$', '', token)
    credentials = google.oauth2.credentials.Credentials(token)

    bq_client = bigquery.Client(project=project_id)
    if use_local_credential:
        bq_client = bigquery.Client(project=project_id, credentials=credentials)
    return bq_client
def plotImp(model, X,figsize, num):
    feature_imp = pd.DataFrame({'Value':model.feature_importance(),'Feature':X.columns})
    plt.figure(figsize=figsize)
    sns.set(font_scale = 1)
    sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value",
                                                        ascending=False)[0:num])
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig('lgbm_importances-01.png')
    plt.show()

In [ ]:
client = get_gcp_bqclient(proj_id, use_local_credential=True)

In [ ]:
# Initializing constants
# client = bigquery.Client(project=proj_id)

# CUST_ID, Bus_Billing_Account_Num, Bus_Prod_Instnc_Id, ADTbT_, ADT_Gazelle_Clients, ADT_FLAG, Base_Snapshot_Date, Security_Origin, Cultural_Segment, Installation_Type, Product_Profile, Province_Grp, Tenure_Month_Groups, ACQUIRED_FROM, DIY_Activated, price_plan, Acquisition_Source, Activation_Ts__Month___Year_, bi_chnl_tag_cd, Channel, Channel__Best_, Channel_Category, CHNL_ORG_TXT__Custom_SQL_Query_, Commitment_Type, Community, Contract_Expiring_Soon, Contract_Type, Cont

trainingSQL = f"""
    select * from `divg-groovyhoon-pr-d2eab4.telus_postpaid_12m_churn_v1.bq_tpc_training_dataset_mob`
"""    #    ='2023-10-01'  order by rand() limit 500000

testSQL = f"""
    select * from `divg-groovyhoon-pr-d2eab4.telus_postpaid_12m_churn_v1.bq_tpc_validation_dataset_mob` 
""" 

In [ ]:
# def data_prep(main_df,scoring_start_date):
#     df=main_df.copy()
#     for col in df.columns:
#         df[col] = list(df[col])
        
#     df = df.T.drop_duplicates().T
#     bool_cols = ['ADTbT_','ADT_Gazelle_Clients','ADT_FLAG','Contracted__Account_','HasFibreTag', 'HasHP', 'HasHSIA',
#                'HasIPTV', 'HasKoodo', 'HasMobility', 'HasPik', 'HasPublic',
#                'HasSMHM', 'HasT_Post', 'HasTOS', 'HasTV', 'HasWHP', 'HasWHSIA',
#                'HP_CNT', 'HS_CNT','TOS_FLAG','Video_','Data_flag','Best_practices'
#             ]
#     date_cols = ['Base_Snapshot_Date']
#     cat_cols = ['Security_Origin','Cultural_Segment','Installation_Type',
#                 'Product_Profile','Province_Grp','Tenure_Month_Groups','Price_Plan_Txt__SHS_',
#                'ACQUIRED_FROM','DIY_Activated','price_plan','Acquisition_Source',
#                 'Activation_Ts__Month___Year_','bi_chnl_tag_cd','Channel',
#                 'Channel__Best_','Channel_Category','CHNL_ORG_TXT__Custom_SQL_Query_',
#                 'Commitment_Type','Community','Contract_Expiring_Soon','Contract_Type',
#                 'CREDIT_VALUE_CD__group_','Customer_Category','Delivery_Technology',
#                 'Delivery_Technology__Account_','POSTAL_CD__Custom_SQL_Query_','Security_Plan_Group',
#                 'Tenure_Month_Groups__All_','Tenure_Year_Group','TV_Count','community_nm',
#                 'census_subdivision_typ','census_subdivision_desc','prizm_name','social_grp_cd','Security_Plan_Type',
#                 'social_grp_nm','lifestage_cd','lifestage_nm','prov_cd','Segment','order_date','munic_nm','chnl_org_txt',
#                 'prov_state_cd','Sales_Channel_tag','txn_sub_typ_txt','SLS_ACTVY_TXT','PRICE_PLAN_TXT__Custom_SQL_Query_'
               
#                ]
#     num_cols = ['Epp_Discount_Months','HSLTE_TOTAL', 'IPTV_CNT','KBRAND_BAN_CT','Nho_Discount_Months',
#                'avg_income', 'med_income','Login_consistency','Arming_consistency'
               
#                ]
#     num_cols += [col for col in df.columns if '_amt_' in col.lower()]
#     target = ['vol']
    
#     le_dict = {}
#     for col in bool_cols+cat_cols:
#         # print(col)
#         df[col] = df[col].fillna('unknown').astype(str)
#         le = LabelEncoder().fit(df[col])
#         le_dict[col] = le
#         df[col] = le.transform(df[col])
#     for col in num_cols:
#         df[col] = df[col].fillna(df[col].median()).astype(float)

#     other_cols = [col for col in df.columns if col not in bool_cols+date_cols+cat_cols+num_cols+target]
#     df.drop(columns=other_cols,axis=1, inplace=True)

#     uni_cols=[]
#     for col in df.columns:
#         if df[col].nunique()==1:
#             uni_cols.append(col)
#     df.drop(columns=uni_cols,axis=1, inplace=True)
#     return df

In [ ]:
# Pulling data from BigQuery
train_df = client.query(trainingSQL).to_dataframe()
test_df = client.query(testSQL).to_dataframe()

In [ ]:
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['target_ind'])

print(train_df.shape, val_df.shape, test_df.shape)

In [ ]:
id_cols = ['ban', 'subscriber_no']

cat_cols = ['revenue_band', 'payment_mthd', 'ebill_ind', 'dvc_non_telus_ind', 'credit_class', 'contract_type', 
            'province', 'bacct_delinq_ind', 'pref_lang', 'demo_sgname', 'demo_lsname'] ## -6  'postal_code' 

num_cols = ['contract_mth', 'sub_tenure', 'ban_tenure', 'cntrct_start_recency', 'cntrct_end_recency', 'age', 
            'rate_plan_amt', 'having_deposit', 'sub_cnt', 
            
            'easy_pymt_avg', 'net_inv_amt_befr_cr_avg', 'cr_disc_amt_avg', 'net_inv_amt_avg', 
            'ld_amt_avg', 'rm_amt_avg', 'hm_call_cnt_avg', 'rm_call_cnt_avg', 'ld_min_qty_avg', 'artm_min_qty_avg', 
            
            'tot_data_usg_avg', 'hm_data_usg_avg', 'rm_data_usg_avg', 
            
            'android_ind', 'apple_ind', 
            
            'flag_mobilephone', 'flag_wireline', 'flag_connecteddevice', 'flag_watch', 
            
            'sms_dnc_recency', 'em_dnc_recency', 'start_dvc_bal_amt', 
            
            'clk_app_ovrview', 'clk_app_bill', 'clk_app_usage', 'clk_app_changefg', 'clk_app_drawer', 'clk_app_offer', 'clk_app_subslct', 
            'clk_app_plan', 'clk_app_obrding', 'clk_app_recmd', 'clk_app_support', 'clk_web_bill', 'clk_web_profile', 'clk_web_plan', 
            'clk_web_moborder', 'clk_web_phnumber', 'clk_web_deal', 'clk_web_rpc', 
            
            'demo_avg_income', 
            
            'call_count_s', 'dial_duration_s', 'ring_duration_s', 'talk_duration_s', 'hold_duration_s', 'effort_durtn_sec_qty_s', 
            'dial_duration_avg_s', 'ring_duration_avg_s', 'talk_duration_avg_s', 'hold_duration_avg_s', 
            
            'arpu_3m_sub', 'msf_3m_sub', 'overage_3m_sub', 

            'avg_usg_cnt_4w_news', 'avg_usg_dl_4w_news', 'avg_usg_ul_4w_news', 'avg_usg_cnt_4w_sports', 'avg_usg_dl_4w_sports', 
            'avg_usg_ul_4w_sports', 'avg_usg_cnt_4w_tv_and_movies', 'avg_usg_dl_4w_tv_and_movies', 'avg_usg_ul_4w_tv_and_movies', 
            'avg_usg_cnt_4w_gaming', 'avg_usg_dl_4w_gaming', 'avg_usg_ul_4w_gaming', 'avg_usg_cnt_4w_smarthome_automation', 
            'avg_usg_dl_4w_smarthome_automation', 'avg_usg_ul_4w_smarthome_automation', 'avg_usg_cnt_4w_smarthome_security', 
            'avg_usg_dl_4w_smarthome_security', 'avg_usg_ul_4w_smarthome_security', 'avg_usg_cnt_4w_shopping_deals', 
            'avg_usg_dl_4w_shopping_deals', 'avg_usg_ul_4w_shopping_deals', 'avg_usg_cnt_4w_telus', 
            'avg_usg_dl_4w_telus', 'avg_usg_ul_4w_telus', 

            'mkt_msg_shs_cnt', 'mkt_msg_hsic_cnt', 'mkt_msg_ttv_cnt', 'mkt_msg_optik_cnt', 'mkt_msg_sing_cnt', 
            'mkt_msg_tos_cnt', 'mkt_msg_lwc_cnt', 'mkt_msg_wifi_cnt', 'mkt_msg_sws_cnt', 'mkt_msg_mob_cnt', 
            'msg_typ_EM', 'msg_typ_SO', 'msg_typ_TM', 'msg_typ_NO', 'msg_typ_SMS',
            
            'msg_typ_ACQUISITION', 'msg_typ_CBR_FFH', 'msg_typ_HSM_West', 'msg_typ_STRATEGIC_SALE', 'msg_typ_TARGETED_CAMPAIGN', 'tot_msg',
            
            'mnh_ind', 'customer_tenure', 'product_mix_all', 'hsic_count', 'ttv_count', 'sing_count', 'mob_count', 'shs_count', 
            'new_hsic_ind', 'new_ttv_ind', 'new_sing_ind', 'new_c_ind', 'new_smhm_ind', 
            
            'ffh_amg_avg', 
            
            # 'frequency', 'recency', 'have_called', 'saved_frequency', 'cancel_frequency', 'saved_rate', 'cancel_rate', 
            
            'mob_frequency', 'mob_recency', 'mob_have_called', 'ffh_frequency', 'ffh_recency', 'ffh_have_called'
            ]

target = ['target_ind']

In [ ]:
def preprocess(_df, id_cols, cat_cols, num_cols, target): 
    df = _df.copy()
    le_dict = {}
    uni_cols=[]
    other_cols = []
    for col in cat_cols:
        df[col] = df[col].fillna('unknown').astype(str)
        le = LabelEncoder().fit(df[col])
        le_dict[col] = le
        df[col] = le.transform(df[col])
    
    other_cols = [col for col in df.columns if col not in cat_cols+num_cols+target]
    print(f'other_cols: {other_cols}')
    
    for col in df.columns:
        if df[col].nunique()==1:
            uni_cols.append(col)

    print(f'uni_cols: {uni_cols}')
            
    drop_cols = [col for col in other_cols+uni_cols if col not in ['ban', 'subscriber_no']] 
    print(f'drop_cols: {drop_cols}')
    
    num_cols_final = [col for col in num_cols if col not in drop_cols]
    
    for col in num_cols_final:
        df[col] = df[col].fillna(df[col].median()).astype(float)
    
    return df, drop_cols

train_df_pre, train_drop_cols = preprocess(train_df, id_cols, cat_cols, num_cols, target)
val_df_pre, val_drop_cols = preprocess(val_df, id_cols, cat_cols, num_cols, target)
test_df_pre, test_drop_cols = preprocess(test_df, id_cols, cat_cols, num_cols, target)

drop_cols = train_drop_cols+val_drop_cols+test_drop_cols

print(drop_cols)

In [ ]:
train_df_pre[target].value_counts()/len(train_df_pre)

In [ ]:
features = [col for col in train_df_pre.columns if col not in drop_cols+id_cols+target]
len(features)

In [ ]:
id_train = train_df_pre[id_cols]
X_train = train_df_pre[features]
y_train = list(np.squeeze(train_df_pre[target].values))

id_val = val_df_pre[id_cols]
X_val = val_df_pre[features]
y_val = list(np.squeeze(val_df_pre[target].values))

id_test = test_df_pre[id_cols]
X_test = test_df_pre[features]
y_test = list(np.squeeze(test_df_pre[target].values))

print(X_train.shape, X_val.shape, X_test.shape)

In [ ]:
sum(y_val)

In [ ]:
sum(y_test)

In [ ]:
def get_lift(prob, y_test, q):
    result = pd.DataFrame(columns=['Prob', 'Churn'])
    result['Prob'] = prob
    result['Churn'] = y_test
    # result['Decile'] = pd.qcut(1-result['Prob'], 10, labels = False)
    result['Decile'] = pd.qcut(result['Prob'], q, labels=[i for i in range(q, 0, -1)])
    add = pd.DataFrame(result.groupby('Decile')['Churn'].mean()).reset_index()
    add.columns = ['Decile', 'avg_real_churn_rate']
    churn_by_decile = pd.DataFrame(result.groupby('Decile')['Churn'].sum()).reset_index() 
    churn_by_decile.columns = ['Decile', 'churn_by_decile']
    result = result.merge(add, on='Decile', how='left')
    result.sort_values('Decile', ascending=True, inplace=True)
    lg = pd.DataFrame(result.groupby('Decile')['Prob'].mean()).reset_index()
    lg.columns = ['Decile', 'avg_model_pred_churn_rate']
    lg.sort_values('Decile', ascending=False, inplace=True)
    lg['avg_churn_rate_total'] = result['Churn'].mean()
    lg['total_churn'] = result['Churn'].sum()
    lg = lg.merge(add, on='Decile', how='left')
    lg['lift'] = lg['avg_real_churn_rate'] / lg['avg_churn_rate_total']
    lg['total_base'] = len(result)
    lg['decile_base'] = len(result)/q
    lg = lg.merge(churn_by_decile, on='Decile', how='left')
    
    return lg

### XGBoost

In [ ]:
# build model and fit in training data
import xgboost as xgb
from sklearn.metrics import roc_auc_score

xgb_model = xgb.XGBClassifier(
    learning_rate=0.05,
    n_estimators=250,
    max_depth=9,
    min_child_weight=1,
    gamma=0,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='binary:logistic',
    nthread=4,
    scale_pos_weight=1,
    seed=27
)

xgb_model.fit(X_train, y_train)
print('xgb training done')

In [ ]:
#predictions on X_val
pred_probs_lgb = xgb_model.predict_proba(X_val, ntree_limit=xgb_model.best_iteration)[:, 1]

lg = get_lift(pred_probs_lgb,y_val,10)
sns.barplot(x='Decile',y='lift',data=lg)
lg

In [ ]:
#predictions on X_test
y_pred = xgb_model.predict_proba(X_test, ntree_limit=xgb_model.best_iteration)[:, 1]

lg = get_lift(y_pred,y_test,10)
sns.barplot(x='Decile',y='lift',data=lg)
lg

### LightGBM

In [ ]:
train_data = lgb.Dataset(X_train, label=y_train)
val_data = lgb.Dataset(X_val, label=y_val)
test_data = lgb.Dataset(X_test, label=y_test)

# del X_train, X_val, df_train, df_test, df_val
gc.collect()
# --- params original
# param = {'n_jobs':4,
#         'learning_rate':0.05,
#         'objective':'binary',
#         'num_leaves':32,
#         'num_threads':4,
#         'max_depth':4,
#         'tree_learner':'serial',
#         'feature_fraction':0.7,
#         'bagging_freq':5,
#         'bagging_fraction':0.7,
#         'verbose':100,
#         "verbosity": -1,
#         'bagging_seed':11,
#         'metric': ['auc', 'binary_logloss'],
#         }

# ------ params for new
param = {'n_jobs':4,
        'learning_rate':0.05,
        'objective':'binary',
        'num_leaves':32,
        'num_threads':4,
        'max_depth':4,
        'tree_learner':'serial',
        'feature_fraction':0.7,
        'bagging_freq':5,
        'bagging_fraction':0.8,
        'verbose':100,
        "verbosity": -1,
        'bagging_seed':11,
        'metric': ['auc', 'binary_logloss'],
        }
num_round  = 100000
model_lgb = lgb.train(param,train_data, num_round, valid_sets=[train_data,val_data],callbacks=[lgb.early_stopping(stopping_rounds=200)])


## evaluate model performance



### use test set: 
features: 2023-10-31 

targets: 2023-11-01 to 2024-06-20

In [ ]:
def get_lift(prob, y_test, q):
    result = pd.DataFrame(columns=['Prob', 'Churn'])
    result['Prob'] = prob
    result['Churn'] = y_test
    # result['Decile'] = pd.qcut(1-result['Prob'], 10, labels = False)
    result['Decile'] = pd.qcut(result['Prob'], q, labels=[i for i in range(q, 0, -1)])
    add = pd.DataFrame(result.groupby('Decile')['Churn'].mean()).reset_index()
    add.columns = ['Decile', 'avg_real_churn_rate']
    churn_by_decile = pd.DataFrame(result.groupby('Decile')['Churn'].sum()).reset_index() 
    churn_by_decile.columns = ['Decile', 'churn_by_decile']
    result = result.merge(add, on='Decile', how='left')
    result.sort_values('Decile', ascending=True, inplace=True)
    lg = pd.DataFrame(result.groupby('Decile')['Prob'].mean()).reset_index()
    lg.columns = ['Decile', 'avg_model_pred_churn_rate']
    lg.sort_values('Decile', ascending=False, inplace=True)
    lg['avg_churn_rate_total'] = result['Churn'].mean()
    lg['total_churn'] = result['Churn'].sum()
    lg = lg.merge(add, on='Decile', how='left')
    lg['lift'] = lg['avg_real_churn_rate'] / lg['avg_churn_rate_total']
    lg['total_base'] = len(result)
    lg['decile_base'] = len(result)/q
    lg = lg.merge(churn_by_decile, on='Decile', how='left')
    
    return lg
    
def calculate_lift(y_true, y_pred, n_bins=10):
    # Create a DataFrame with y_true and y_pred
    data = pd.DataFrame({'y_true': y_true, 'y_pred': y_pred})

    # Sort the DataFrame by y_pred in descending order
    data = data.sort_values(by='y_pred', ascending=False)

    # Calculate the base rate
    base_rate = np.mean(y_true)

    # Calculate the lift
    data['cumulative_positive'] = data['y_true'].cumsum()
    data['cumulative_total'] = np.arange(1, len(data) + 1)
    data['cumulative_rate'] = data['cumulative_positive'] / data['cumulative_total']
    data['lift'] = data['cumulative_rate'] / base_rate

    # Calculate lift for each bin
    data['bin'] = pd.qcut(data['cumulative_total'], n_bins, labels=False)
    lift_by_bin = data.groupby('bin')['lift'].mean()

    return lift_by_bin


### Model Eval on X_val

In [ ]:
pred_probs_lgb = model_lgb.predict(X_val,num_iteration=model_lgb.best_iteration)

In [ ]:
lg = get_lift(pred_probs_lgb,y_val,10)
sns.barplot(x='Decile',y='lift',data=lg)
lg

# lg.to_csv('gs://divg-groovyhoon-pr-d2eab4-default/downloads/decile_lift_v2_lgbm.csv', index=False)

In [ ]:
lg = get_lift(pred_probs_lgb,y_val,100)
sns.barplot(x='Decile',y='lift',data=lg)
lg

# lg.to_csv('gs://divg-groovyhoon-pr-d2eab4-default/downloads/percentile_lift_v2_lgbm.csv', index=False)

### Export df_val_exp_v2_lgbm

In [ ]:
#join ban_test, X_test, y_test and y_pred and print to csv
#CHECK THE SIZE OF EACH COMPONENT BEFORE JOINING
q=10
df_ban_val = id_val
df_val_exp = df_ban_val.join(X_val) 
df_val_exp['y_val'] = y_val
df_val_exp['y_pred_proba'] = pred_probs_lgb
df_val_exp['y_pred'] = (df_val_exp['y_pred_proba'] > 0.5).astype(int)
df_val_exp['decile'] = pd.qcut(df_val_exp['y_pred_proba'], q, labels=[i for i in range(q, 0, -1)])
df_val_exp['decile'] = df_val_exp['decile'].astype(int)

df_val_exp.info()

df_val_exp_final = pd.DataFrame()
df_val_exp_final['ban'] = df_val_exp['ban']
df_val_exp_final['subscriber_no'] = df_val_exp['subscriber_no']
df_val_exp_final['score'] = df_val_exp['y_pred_proba'] 
df_val_exp_final['decile'] = pd.qcut(df_val_exp_final['score'], 10, labels=[i for i in range(q, 0, -1)])
# df_val_exp_final['percentile'] = pd.qcut(df_val_exp_final['score'], 100, labels=[i for i in range(q, 0, -1)])

df_val_exp_final.to_csv('gs://divg-groovyhoon-pr-d2eab4-default/downloads/df_val_exp_final_v2_lgbm.csv', index=False)
print("....df_val_exp_final exported")

### Model Eval on X_test

In [ ]:
y_pred = model_lgb.predict(X_test,num_iteration=model_lgb.best_iteration)

In [ ]:
lg = get_lift(y_pred,y_test,10)
sns.barplot(x='Decile',y='lift',data=lg)
lg

### Create df_test_exp to create `divg-groovyhoon-pr-d2eab4.telus_postpaid_12m_churn_v1.bq_telus_postpaid_12m_churn_scores`

In [ ]:
#join ban_test, X_test, y_test and y_pred and print to csv
#CHECK THE SIZE OF EACH COMPONENT BEFORE JOINING
q=10
df_ban_test = id_test
df_test_exp = df_ban_test.join(X_test) 
df_test_exp['y_test'] = y_test
df_test_exp['y_pred_proba'] = y_pred
df_test_exp['y_pred'] = (df_test_exp['y_pred_proba'] > 0.5).astype(int)
df_test_exp['decile'] = pd.qcut(df_test_exp['y_pred_proba'], q, labels=[i for i in range(q, 0, -1)])
df_test_exp['decile'] = df_test_exp['decile'].astype(int)

df_test_exp.info()

df_test_exp_final = pd.DataFrame()
df_test_exp_final['ban'] = df_test_exp['ban']
df_test_exp_final['subscriber_no'] = df_test_exp['subscriber_no']
df_test_exp_final['score'] = df_test_exp['y_pred_proba'] 
df_test_exp_final['decile'] = pd.qcut(df_test_exp_final['score'], 10, labels=[i for i in range(q, 0, -1)])
# df_test_exp_final['percentile'] = pd.qcut(df_test_exp_final['score'], 100, labels=[i for i in range(q, 0, -1)])

df_test_exp_final.to_csv('gs://divg-groovyhoon-pr-d2eab4-default/downloads/df_test_exp_final_v2_lgbm.csv', index=False)
print("....df_test_exp_final exported")

In [ ]:
def get_gcp_bqclient(project_id, use_local_credential=False):
    token = os.popen('gcloud auth print-access-token').read()
    token = re.sub(f'\n$', '', token)
    credentials = google.oauth2.credentials.Credentials(token)

    bq_client = bigquery.Client(project=project_id)
    if use_local_credential:
        bq_client = bigquery.Client(project=project_id, credentials=credentials)
    return bq_client

def plotImp(model, X,figsize, num):
    feature_imp = pd.DataFrame({'Value':model.feature_importance(),'Feature':X.columns})
    plt.figure(figsize=figsize)
    sns.set(font_scale = 1)
    sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value",
                                                        ascending=False)[0:num])
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig('lgbm_importances-01.png')
    plt.show()

In [ ]:
# import time

# client = get_gcp_bqclient(proj_id, use_local_credential=True)

# # create temp table
# project_id = 'divg-groovyhoon-pr-d2eab4' 
# table_name = 'divg-groovyhoon-pr-d2eab4.telus_postpaid_12m_churn_v1.bq_telus_postpaid_12m_churn_scores' 
# table_def = bigquery.Table(table_name)
# table = client.create_table(table_def)
# time.sleep(2)
# print(
#     f'Created score table {table.project}.{table.dataset_id}.{table.table_id}'
# )

# insert data to temp table
config = bigquery.LoadJobConfig()
config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
job = client.load_table_from_dataframe(df_test_exp_final, table_def, job_config=config)
job.result()  # wait for the job to complete.
time.sleep(2)
table = client.get_table(table_def)
print(
    f'Loaded {table.num_rows} rows and {len(table.schema)} columns to {table_def}'
)

In [ ]:
error

In [ ]:
df_test_exp.to_csv('gs://divg-groovyhoon-pr-d2eab4-default/downloads/df_test_exp.csv', index=False)
print("....df_test_exp exported")

In [ ]:
df_test_exp_1r = df_test_exp.sample(n=1)
df_test_exp_1r.to_csv('gs://divg-groovyhoon-pr-d2eab4-default/downloads/df_test_exp_1r.csv', index=False)

In [ ]:
client = get_gcp_bqclient(proj_id, use_local_credential=True)

In [ ]:
from google.cloud import bigquery
import pandas as pd 
import numpy as np 

def export_dataframe_to_bq(df, client, table_id='', schema_list=[], generate_schema=True, write='overwrite'): 
    
    """
    Load the input dataframe to a table in bigquery. 
    
    Args: 
        df (DataFrame): A DataFrame that you want to export to BQ.
        client: A BigQuery client instance. e.g. client = bigquery.Client(project=project_id). 
        table_id (string): A string with dataset and table name. i.e 'ttv_churn_dataset.bq_tv_churn_score'. 
        schema_list (list of string, optional): List of the SchemaFields if provided, otherwise the function can generate it for you. 
        generate_schema (bool, optional): True (if True, the function will provide schema for you) or False (provide your own list) 
        write (string, optional):  if 'overwrite', the function will overwrite the existing table
                                   if 'append', it will append to the existing table 
                                   
    Returns: 
        None 
        
    Example: 
        export_dataframe_to_bq(final_df, bq_client, 'call_to_retention_dataset.bq_call_to_retention_scores_temp')
        
    """

    dtype_bq_mapping = { 
        np.dtype('int64'): 'INTEGER', 
        np.dtype('float64'): 'FLOAT', 
        np.dtype('float32'): 'FLOAT', 
        np.dtype('object'): 'STRING', 
        np.dtype('bool'): 'BOOLEAN', 
        np.dtype('datetime64[ns]'): 'DATE', 
        pd.Int64Dtype(): 'INTEGER' 
    } 
    
    if write == 'overwrite': 
        write_type = 'WRITE_TRUNCATE' 
    else: 
        write_type = 'WRITE_APPEND' 
        
    if len(schema_list) == 0: 
        generate_schema = True 
    else: 
        generate_schema = False
        
    try: 
        if generate_schema == True: 
            schema_list = [] 
            for column in df.columns: 
                schema_list.append(bigquery.SchemaField(column, dtype_bq_mapping[df.dtypes[column]], mode='NULLABLE')) 

        # Sending to bigquery 
        job_config = bigquery.LoadJobConfig(schema=schema_list, write_disposition=write_type) 
        job = client.load_table_from_dataframe(df, table_id, job_config=job_config) 
        job.result() 
        table = client.get_table(table_id) # Make an API request 
        print("Loaded {} rows and {} columns to {}".format(table.num_rows, len(table.schema), table_id)) 

    except NameError as e: 
        print(f"Error : {e}")

In [ ]:
export_dataframe_to_bq(df_test_exp, client, table_id='telus_postpaid_12m_churn_v1.bq_telus_postpaid_12m_churn_scores', generate_schema=True, write='overwrite')

In [ ]:
plotImp(model_lgb, X_train,figsize=(15,8), num=50)

In [ ]:
error

### predict on Nov and Dec clients and evaluate with Jan the same clients

In [ ]:
sql_1 = f"""
select distinct * except(invol, churn,Sales_Agent_ID,src_usr_chnl_txt,order_date,POSTAL_CD__Custom_SQL_Query_) from `divg-team-v03-pr-de558a.shs_churn_model.shs_churn_data_final_Nov_2023`
     where  Base_Snapshot_Date = '2023-11-01' and security_origin='Organic'
"""

sql_2 = f"""
select distinct * except(invol, churn,Sales_Agent_ID,src_usr_chnl_txt,order_date,POSTAL_CD__Custom_SQL_Query_) from `divg-team-v03-pr-de558a.shs_churn_model.shs_churn_data_final_Dec_2023`
     where  Base_Snapshot_Date = '2023-12-01' and security_origin='Organic'
"""

sql_3 = f"""
select distinct * except(invol, churn,Sales_Agent_ID,src_usr_chnl_txt,order_date,POSTAL_CD__Custom_SQL_Query_) from `divg-team-v03-pr-de558a.shs_churn_model.shs_churn_data_final_Jan_2024` 
     where  Base_Snapshot_Date = '2024-01-01' and security_origin='Organic'
"""

In [ ]:
sql_1 = f"""
select distinct * except(invol, churn,Sales_Agent_ID,src_usr_chnl_txt,order_date,POSTAL_CD__Custom_SQL_Query_) from `divg-team-v03-pr-de558a.shs_churn_model.shs_churn_data_final_Nov_2023`
     where  Base_Snapshot_Date = '2023-11-01'
"""

sql_2 = f"""
select distinct * except(invol, churn,Sales_Agent_ID,src_usr_chnl_txt,order_date,POSTAL_CD__Custom_SQL_Query_) from `divg-team-v03-pr-de558a.shs_churn_model.shs_churn_data_final_Dec_2023`
     where  Base_Snapshot_Date = '2023-12-01'
"""

sql_3 = f"""
select distinct * except(invol, churn,Sales_Agent_ID,src_usr_chnl_txt,order_date,POSTAL_CD__Custom_SQL_Query_) from `divg-team-v03-pr-de558a.shs_churn_model.shs_churn_data_final_Jan_2024` 
     where  Base_Snapshot_Date = '2024-01-01'
"""

In [ ]:
client = get_gcp_bqclient(proj_id, use_local_credential=True)

In [ ]:
# df_nov = client.query(sql_1).to_dataframe()
df_dec = client.query(sql_2).to_dataframe()
df_jan= client.query(sql_3).to_dataframe()

In [ ]:
# df_nov = df_nov.drop_duplicates(subset=['Bus_Billing_Account_Num'], keep='first')
df_dec = df_dec.drop_duplicates(subset=['Bus_Billing_Account_Num'], keep='first')
df_jan = df_jan.drop_duplicates(subset=['Bus_Billing_Account_Num'], keep='first')

In [ ]:
# df_nov.shape
df_dec.shape,df_jan.shape

In [ ]:
# df_nov['Bus_Billing_Account_Num'].nunique()==len(df_nov),
df_dec['Bus_Billing_Account_Num'].nunique()==len(df_dec),df_jan['Bus_Billing_Account_Num'].nunique()==len(df_jan)

In [ ]:
for col in df_dec.columns:
    # df_nov[col] = list(df_nov[col])
    df_dec[col] = list(df_dec[col])
    df_jan[col] = list(df_jan[col])

In [ ]:
# # ban_nov = list(df_nov['Bus_Billing_Account_Num'].unique())
# ban_dec = list(df_dec['Bus_Billing_Account_Num'].unique())
# ban_jan = list(df_jan['Bus_Billing_Account_Num'].unique())

# # ban_common = set(ban_nov).intersection(set(ban_dec))
# ban_common = set(ban_dec)
# ban_common = set(ban_common).intersection(set(ban_jan))

# # ban_common = [ban for ban in ban_nov if ban in ban_dec and ban in ban_jan]
# # len(ban_nov),len(ban_dec),len(ban_jan),

# len(ban_common)

In [ ]:
# # df_nov = df_nov.loc[df_nov['Bus_Billing_Account_Num'].isin(list(ban_common))]
# df_dec = df_dec.loc[df_dec['Bus_Billing_Account_Num'].isin(list(ban_common))]
# df_jan = df_jan.loc[df_jan['Bus_Billing_Account_Num'].isin(list(ban_common))]
# # len(df_nov),len(df_dec),len(df_jan)

# # len(df_nov),
# len(df_dec),len(df_jan)

In [ ]:
# df_dec=df_dec.merge(df_jan[['Bus_Billing_Account_Num','vol']],on='Bus_Billing_Account_Num',how='left')

In [ ]:
# df_dec['vol_x'].value_counts()

In [ ]:
# df_dec['vol_y'].value_counts()

In [ ]:
# # y_true_nov = list(np.squeeze(df_nov['vol_x'].values)) #nov label
# # y_true_dec = list(np.squeeze(df_nov['vol_y'].values))  # dec label

# y_true_jan = list(np.squeeze(df_dec['vol_y'].values))  # jan label

# # len(y_true_nov),len(y_true_dec),

# len(y_true_jan)

In [ ]:
y_true_dec = list(np.squeeze(df_dec['vol'].values)) 
y_true_jan = list(np.squeeze(df_jan['vol'].values))

print(len(y_true_dec))
print(len(y_true_jan))

In [ ]:
le_dict

In [ ]:
le_dict_cp = le_dict.copy()

In [ ]:
def preprocessing(df,le_dict_cp):
    df_ = df.copy()
    for col in bool_cols+cat_cols:
        # if col in ['order_date','price_plan','POSTAL_CD__Custom_SQL_Query_','std_emp_cnt_r30d_smhm','std_talk_time_r30d_smhm', 'livingwell_cnt', 'facilities_cnt', 'upgr_wln_security_cnt_r30d','streamplus_cnt']: continue
        print(col, end=' ')
        df_[col] = df_[col].fillna('unknown').astype(str)
        # a = set(np.array(le_dict[col].classes_))
        a = set(le_dict_cp[col].classes_)
        # print('before',a)        
        b = set(df_[col].unique())
        new_x = a^b
        a.update(new_x)
        le_dict_cp[col].classes_ = np.array(a)
        df_[col] = le_dict_cp[col].fit_transform(df_[col])
    
    for col in num_cols:
        df_[col] = df_[col].fillna(df_[col].median()).astype(float)
        
    other_cols = [col for col in df_.columns if col not in bool_cols+cat_cols+num_cols+target]
     # and col not in ['call_to_retention_ind','vol','vol_ctr_ind','std_emp_cnt_r30d_smhm','std_talk_time_r30d_smhm', 'livingwell_cnt', 'facilities_cnt', 'upgr_wln_security_cnt_r30d','streamplus_cnt']
    other_cols = [col for col in other_cols if col not in features]
    df_.drop(columns=other_cols,axis=1, inplace=True)
    uni_cols=[]
    for col in df_.columns:
        # if col in ['call_to_retention_ind','vol','vol_ctr_ind','std_emp_cnt_r30d_smhm','std_talk_time_r30d_smhm', 'livingwell_cnt', 'facilities_cnt', 'upgr_wln_security_cnt_r30d','streamplus_cnt']: continue
        if df_[col].nunique()==1:
            uni_cols.append(col)
    uni_cols = [col for col in uni_cols if col not in features]
    df_.drop(columns=uni_cols,axis=1, inplace=True)  
    return df_

In [ ]:
# df_nov_processed = preprocessing(df_nov,le_dict_cp)

df_dec_processed = preprocessing(df_dec,le_dict_cp)
df_jan_processed = preprocessing(df_jan,le_dict_cp)

In [ ]:
# X_test_nov = df_nov_processed[features]

X_test_dec = df_dec_processed[features]
X_test_jan = df_jan_processed[features]

In [ ]:
# y_pred_nov = model_lgb.predict(X_test_nov,num_iteration=model_lgb.best_iteration)
y_pred_dec = model_lgb.predict(X_test_dec,num_iteration=model_lgb.best_iteration)
y_pred_jan = model_lgb.predict(X_test_jan,num_iteration=model_lgb.best_iteration)

In [ ]:
# lg_1 = get_lift(y_pred_nov,y_true_jan,10)
# sns.barplot(x='Decile',y='lift',data=lg_1)
# lg_1

In [ ]:
lg_2 = get_lift(y_pred_dec,y_true_dec,20)
sns.barplot(x='Decile',y='lift',data=lg_2)
lg_2

In [ ]:
lg_2 = get_lift(y_pred_dec,y_true_dec,10)
sns.barplot(x='Decile',y='lift',data=lg_2)
lg_2

In [ ]:
lg_3 = get_lift(y_pred_jan,y_true_jan,10)
sns.barplot(x='Decile',y='lift',data=lg_3)
lg_3

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
df_ = X_train.copy()
df_['target'] = y_train
corr_df = df_.corr()['target']
corr_df=corr_df.sort_values(ascending=False)
corr_df

In [ ]:
corr_df.head(20)

In [ ]:
feature_imp = pd.DataFrame({'Value':model_lgb.feature_importance(),'Feature':X_train.columns})
feature_imp.sort_values(by="Value", ascending=False,inplace=True)

In [ ]:
importances = model_lgb.feature_importance()

# Get the index of importances from greatest importance to least
sorted_index = np.argsort(importances)[::-1]
x = range(len(importances))

feature_names = X_train.columns

# Create tick labels 
labels = np.array(feature_names)[sorted_index]

for idx in sorted_index: 
    print(f'{feature_names[idx]}, {importances[idx]}', end='\n')

In [ ]:
feature_imp.loc[feature_imp.Value>30].head(20)

In [ ]:
feature_imp.loc[feature_imp.Value>60]['Feature'].values